In [2]:
import os
import numpy as np
import pickle
import h5py
import scipy.signal as sps
import scipy.interpolate as spi
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from feature_editing import FeatureDesigner
from dataset_parsing import DataInfo, DataProducer


SyntaxError: non-default argument follows default argument (feature_editing.py, line 100)

In [47]:
import xgboost as xgb
import h5py
f = h5py.File('/home/tim/repos/wesad-tools/data/formatted_data_feat.h5', 'r')
x = np.array([[1,2,3],[4,5,6]]).T; y = np.array([[7,8,9,10],[11,12,13,14]]).T


np.vstack
ap = np.concatenate((x,y))
print(ap)
a = np.array([])
# a.concatenate(ap)

xx = [1,2,3]
yy = [4,5,6]
zz = [7,8,9]
aa = np.array([xx, yy, zz]).T
print(aa)

[[ 1  4]
 [ 2  5]
 [ 3  6]
 [ 7 11]
 [ 8 12]
 [ 9 13]
 [10 14]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]


In [5]:
print(f['S2'].keys())
print(len(f['S2']['chestACCMean']))
sz = min(len(f['S2']['chestACCMean']), len(f['S3']['chestACCMean']))
test_subject = 'S2'
train_subjects = [sub for sub in f.keys() if sub.startswith('S') and sub != test_subject]
features = [ 
    'chestACCMean',
    'chestACCStd',
    # 'chestACC0Mean',
    # 'chestACC0Std',
    # 'chestACC0Freq',
    # 'chestACC1Mean',
    # 'chestACC1Std',
    # 'chestACC1Freq',
    # 'chestACC2Mean',
    # 'chestACC2Std',
    # 'chestACC2Freq',
    'chestECGMeanHR',
    'chestECGStdHR',
    'chestECGMeanHRV',
    'chestECGStdHRV',
    # 'chestECGTINN',
    # 'chestECGRMS',
    'chestEMGMean',
    'chestEMGStd',
    'chestEMGFq',
    'chestEDAMean',
    # 'chestEDAMax',
    # 'chestEDAMin',
    # 'chestEDADyn',
    'chestTempMean',
    # 'chestTempMax',
    # 'chestTempMin',
    # 'chestTempDyn',
    # 'chestResp',
    'wristACCMean',
    'wristACCMax',
    'wristACCMin',
    'wristBVPMeanHR',
    'wristBVPStdHR',
    'wristBVPMeanHRV',
    'wristBVPStdHRV',
    # 'wristBVPTINN',
    # 'wristBVPRMS',
    'wristEDAMean',
    # 'wristEDAMax',
    # 'wristEDAMin',
    # 'wristEDADyn',
    'wristTEMPMean',
    # 'wristTEMPMax',
    # 'wristTEMPMin',
    # 'wristTEMPDyn'
    ]
train_arr = np.array([])
for sub in train_subjects:
    print([(ftr,np.array(f[sub][ftr][:]).shape) for ftr in features])
    sub_train_arr = np.array([np.array(f[sub][ftr][:]) for ftr in features]).T
    print(np.shape(sub_test_arr))
    train_arr = np.concatenate((test_arr, sub_test_arr))
train_labels = np.array([0 if i==1 else 1 for i in f['S2']['label'][:sz]])
test_labels = np.array([0 if i==1 else 1 for i in f['S3']['label'][:sz]])
test_arr.shape


<KeysViewHDF5 ['chestACC0Freq', 'chestACC0Mean', 'chestACC0Std', 'chestACC1Freq', 'chestACC1Mean', 'chestACC1Std', 'chestACC2Freq', 'chestACC2Mean', 'chestACC2Std', 'chestACCMean', 'chestACCStd', 'chestECGMeanHR', 'chestECGMeanHRV', 'chestECGRMS', 'chestECGStdHR', 'chestECGStdHRV', 'chestECGTINN', 'chestEDADyn', 'chestEDAMax', 'chestEDAMean', 'chestEDAMin', 'chestEMGFq', 'chestEMGMean', 'chestEMGStd', 'chestResp', 'chestTempDyn', 'chestTempMax', 'chestTempMean', 'chestTempMin', 'label', 'wristACCMax', 'wristACCMean', 'wristACCMin', 'wristBVPMeanHR', 'wristBVPMeanHRV', 'wristBVPRMS', 'wristBVPStdHR', 'wristBVPStdHRV', 'wristBVPTINN', 'wristEDADyn', 'wristEDAMax', 'wristEDAMean', 'wristEDAMin', 'wristTEMPDyn', 'wristTEMPMax', 'wristTEMPMean', 'wristTEMPMin']>
4255300
[('chestACCMean', (4545100,)), ('chestACCStd', (4545100,)), ('chestECGMeanHR', (4550000,)), ('chestECGStdHR', (4550000,)), ('chestECGMeanHRV', (4550000,)), ('chestECGStdHRV', (4550000,)), ('chestEMGMean', (4545100,)), ('ches

NameError: name 'sub_test_arr' is not defined

In [4]:
x_train = xgb.DMatrix(train_arr, label=train_labels)
x_test = xgb.DMatrix(test_arr, label=test_labels)
num_round = 10
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(x_train, 'eval'), (x_test, 'train')]
bst = xgb.train(param, x_train, num_round, evallist)

ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (80,))

In [5]:
import os
import numpy as np
import pickle
import h5py
from datetime import datetime
from feature_editing import FeatureDesigner
import time

class DataInfo:
    """
    Data information from readme.
    
    :param: data_idx = sample labels
    :param: sampling_freq_hz = sampling frequency (hz) of data signals
    """
    data_idx = ['2', '3', '4', '5', '6',]
                # '7', '8', '9', '10', '11',
                # '13', '14', '15', '16', '17']
    # data_idx = ['2','3']
    label_data_fs = 700
    fs_hz = { #see README
        'chest' : {
            signal : 700 for signal in ['ACC', 'ECG', 'EMG',
                                        'EDA', 'Temp', 'Resp']
        },
        'wrist' : {
            'ACC'  : 32,
            'BVP'  : 64,
            'EDA'  : 4,
            'TEMP' : 4
        }
    }

    def __init__(self):
        now = datetime.now()
        self.date = now.strftime("%d/%m/%Y %H:%M:%S")


class DataManager:
    def __init__(self, data_filepath, data_info):
        self.data_filepath = data_filepath
        self.data_info = data_info
        self.designer = FeatureDesigner(data_info)
        with open(data_filepath, 'rb') as f:
            raw_data = pickle.load(f, encoding='latin1' )

    def parse(self):
        feat_vecs = {}
        for component in ([('chest', feat) for feat in [ 'ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']] + 
                          [('wrist', feat) for feat in ['ACC', 'BVP', 'EDA', 'TEMP']]):
        # for component in [('wrist', feat) for feat in ['ACC', 'BVP', 'EDA', 'TEMP']]:
            print(f'Parsing {component[0]}{component[1]} data')
            feat_vecs.update( self.designer.edit_feature(f'{component[0]}{component[1]}',
                                                         self.data['signal'][component[0]][component[1]]) )
        return feat_vecs

if __name__ == '__main__':

    data_info = DataInfo()
    for idx in [2]:#data_info.data_idx:
        data_path = os.path.abspath(os.path.join(os.path.dirname( __file__ ),
                                                '..',
                                                'data',
                                                'WESAD',
                                                f'S{idx}',
                                                f'S{idx}.pkl'))
        print(f'Parsing {data_path}')
        t0 = time.time()
        grp = fout.create_group(f'S{idx}')
        dp = DataManager(data_path, data_info)


S2
['S3', 'S4', 'S5', 'S6']


In [4]:
import numpy as np
import matplotlib.pyplot as plt



In [17]:

def upsample_data(signal, fs, custom_ts):
    time_signal = np.arange(0, len(signal)*(1/fs), 1/fs)
    f = spi.interp1d(time_signal, signal, kind='zero', fill_value='extrapolate')
    return f(custom_ts)

def rolling_skip(data, fs, win_sz, skip=1, apply=None , **kwargs):
    res = np.zeros(len(range(0, len(data), skip)))
    for idx, i in enumerate(range(0, len(data), skip)):
        try:
            if apply is None:
                res[idx] = data[i]
            else:
                res[idx] = apply(data[i+win_sz], **kwargs)
        except ValueError:
            res[idx] = res[idx-1]
    return upsample_data(res, fs/skip, np.arange(0, len(data)*(1/fs), 1/fs))

def fna(data, **kwargs):
    print(x, kwargs)
    return

skip = 3
fs = 2.3
nx = 33
win_sz = 2
x = np.arange(0, nx, 1/fs)

a = np.array([xi**2 for xi in x])

b = rolling_skip(a, fs, win_sz, skip=3, apply=fna, kwargs = {'kw' : 'val'})



08695652 26.52173913 26.95652174 27.39130435 27.82608696 28.26086957
 28.69565217 29.13043478 29.56521739 30.         30.43478261 30.86956522
 31.30434783 31.73913043 32.17391304 32.60869565] {'kwargs': {'kw': 'val'}}
[ 0.          0.43478261  0.86956522  1.30434783  1.73913043  2.17391304
  2.60869565  3.04347826  3.47826087  3.91304348  4.34782609  4.7826087
  5.2173913   5.65217391  6.08695652  6.52173913  6.95652174  7.39130435
  7.82608696  8.26086957  8.69565217  9.13043478  9.56521739 10.
 10.43478261 10.86956522 11.30434783 11.73913043 12.17391304 12.60869565
 13.04347826 13.47826087 13.91304348 14.34782609 14.7826087  15.2173913
 15.65217391 16.08695652 16.52173913 16.95652174 17.39130435 17.82608696
 18.26086957 18.69565217 19.13043478 19.56521739 20.         20.43478261
 20.86956522 21.30434783 21.73913043 22.17391304 22.60869565 23.04347826
 23.47826087 23.91304348 24.34782609 24.7826087  25.2173913  25.65217391
 26.08695652 26.52173913 26.95652174 27.39130435 27.82608696 2

IndexError: index 77 is out of bounds for axis 0 with size 76